In [1]:
import os
import argparse
import numpy as np
from scipy.io import wavfile
import features as mfcc
from hmmlearn import hmm
import sklearn.mixture as skm

In [7]:
def loadtrainingfeatures(input_folder,start=0,end=4):

    digit_features = {}

    for dirname in os.listdir(input_folder):

            digit_folder = os.path.join(input_folder,dirname)

            if not os.path.isdir(digit_folder):
                continue

            all_speaker_features  = []

            for speaker in os.listdir(digit_folder):

                speaker_folder =  os.path.join(digit_folder,speaker)


                features = np.asarray(())

                for filename in [x for x in os.listdir(speaker_folder) if x.endswith('.wav')][:end+1]:

                    filepath = os.path.join(speaker_folder,filename)
                    sampling_rate, audio_signal = wavfile.read(filepath)

                    feat = mfcc.getMFCC(audio_signal,sampling_rate)

                    if features.size == 0:
                        features = feat
                    else:
                        features = np.vstack((features, feat))

                all_speaker_features.append(features)

            digit_features[dirname] = all_speaker_features
    return digit_features

In [8]:
def loadTestData(test_folder, start =0, end= 5):
    speakers_data = {}

    for dirname in os.listdir(test_folder):

            speaker_folder = os.path.join(test_folder,dirname)
            
            speaker_features = {}

            if not os.path.isdir(speaker_folder):
                continue

            for digit in os.listdir(speaker_folder):

                digit_folder =  os.path.join(speaker_folder,digit)
                
                digit_features = []

                features = np.asarray(())

                for filename in [x for x in os.listdir(digit_folder) if x.endswith('.wav')][:5]:

                    filepath = os.path.join(digit_folder,filename)
                    sampling_rate, audio_signal = wavfile.read(filepath)
                    

                    feat = mfcc.getMFCC(audio_signal,sampling_rate)

                    if features.size == 0:
                        features = feat
                    else:
                        features = np.vstack((features, feat))

                digit_features.append(features)
                speaker_features[digit] = digit_features
            speakers_data[dirname] = speaker_features
    return speakers_data

In [9]:
train_folder = "../free-digit-recordings/train_folder"
test_folder = "../free-digit-recordings/test_folder"
digit_features = loadtrainingfeatures(train_folder)
enroll_features = loadTestData(test_folder,start=7,end=12)
test_features = loadTestData(test_folder,start=26,end=27)

In [37]:
all_digit_UBM = {}

for digit in digit_features:
    digit_hmm = hmm.GMMHMM(n_components=8, n_mix = 5, n_iter=100, covariance_type='diag')
#     digit_gmms = []
#     for speaker in digit_features[digit]:
#         speaker_gmm = skm.GaussianMixture(n_components=8,covariance_type='diag')
#         speaker_gmm.fit(speaker)
#         digit_gmms.append(speaker_gmm)
    
#     digit_hmm.gmm_ = digit_gmms
    
    for speaker in digit_features[digit]:
        digit_hmm.fit(speaker)
    all_digit_UBM[digit] = digit_hmm
        

Even though the 'startprob_' attribute is set, it will be overwritten during initialization because 'init_params' contains 's'
Even though the 'transmat_' attribute is set, it will be overwritten during initialization because 'init_params' contains 't'
Even though the 'weights_' attribute is set, it will be overwritten during initialization because 'init_params' contains 'w'
Even though the 'means_' attribute is set, it will be overwritten during initialization because 'init_params' contains 'm'
Even though the 'covars_' attribute is set, it will be overwritten during initialization because 'init_params' contains 'c'
Even though the 'startprob_' attribute is set, it will be overwritten during initialization because 'init_params' contains 's'
Even though the 'transmat_' attribute is set, it will be overwritten during initialization because 'init_params' contains 't'
Even though the 'weights_' attribute is set, it will be overwritten during initialization because 'init_params' contains '

Even though the 'transmat_' attribute is set, it will be overwritten during initialization because 'init_params' contains 't'
Even though the 'weights_' attribute is set, it will be overwritten during initialization because 'init_params' contains 'w'
Even though the 'means_' attribute is set, it will be overwritten during initialization because 'init_params' contains 'm'
Even though the 'covars_' attribute is set, it will be overwritten during initialization because 'init_params' contains 'c'
Even though the 'startprob_' attribute is set, it will be overwritten during initialization because 'init_params' contains 's'
Even though the 'transmat_' attribute is set, it will be overwritten during initialization because 'init_params' contains 't'
Even though the 'weights_' attribute is set, it will be overwritten during initialization because 'init_params' contains 'w'
Even though the 'means_' attribute is set, it will be overwritten during initialization because 'init_params' contains 'm'
E

Even though the 'weights_' attribute is set, it will be overwritten during initialization because 'init_params' contains 'w'
Even though the 'means_' attribute is set, it will be overwritten during initialization because 'init_params' contains 'm'
Even though the 'covars_' attribute is set, it will be overwritten during initialization because 'init_params' contains 'c'
Even though the 'startprob_' attribute is set, it will be overwritten during initialization because 'init_params' contains 's'
Even though the 'transmat_' attribute is set, it will be overwritten during initialization because 'init_params' contains 't'
Even though the 'weights_' attribute is set, it will be overwritten during initialization because 'init_params' contains 'w'
Even though the 'means_' attribute is set, it will be overwritten during initialization because 'init_params' contains 'm'
Even though the 'covars_' attribute is set, it will be overwritten during initialization because 'init_params' contains 'c'
Eve

In [ ]:
all

In [38]:
speaker_models = {}
for s in enroll_features:
    sp_digit_model = {}
    for d in enroll_features[s]:
        
        x = enroll_features[s][d]
        model = all_digit_UBM[d].predict_proba(x[0])
        sp_digit_model[d] = model
        
        #would save ubm for speaker here:
    speaker_models[s] = sp_digit_model


        
        
        

In [40]:
speaker_models['george']['0'].cov


(255, 8)

In [61]:
import pickle
with open("filename.pkl", "wb") as file: pickle.dump(all_digit_UBM, file)
with open("filename.pkl", "rb") as file: pickle.load(file)

In [ ]:
fakeutterance = ['0','5','6','7','9']


for dig in fakeutterance:
    scores ={}
    for digit in all_digit_UBM:

        model = all_digit_UBM[digit]
        score = model.score(test_features['george'][dig])
        scores[digit] = score
    predict = max(scores, key=scores.get)
    print("Test on true label ", dig,": predict result label is ", predict)
    
    


In [ ]:
def getI_vector(N,F,C,T,t_dim):
    
    I = np.eye(t_dim);
    B =  np.dot(np.linalg.pinv(T),S)    
    cov_i = I + ((B*N).dot(T))
    V = np.dot(B,F)
    ivector = np.dot(np.linalg.pinv(cov_i) ,V)
    return ivector

In [ ]:
def score(sim,lenOfutterance = 6):
    score = np.sum(sum)/lenOfutterance
    

In [ ]:
def logLiklihoodRatio():

In [ ]:
def cosineSimilarity():

In [ ]:
def WCC():

In [ ]:
def getN_F_stats(model, features):
    
    n_samples = 
    
    N = np.sum(model)
    

In [58]:
import random

def get_digit_string(length):
    sample_letters = '0123456789'
    result_str = ''.join((random.choice(sample_letters) for i in range(length)))

get_digit_string(6)
get_digit_string(6)

Random string is: 379055
Random string is: 105555
